In [ ]:
# !pip install --upgrade torch torchvision
import sys
sys.path.insert(1, '../input/utility-scripts')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import DenseNet3D
import Simple_3dCNN

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import random
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn import model_selection as sk_model_selection

import transforms3D


In [ ]:
# class FocalLoss(nn.Module):

#     def __init__(self, gamma=0, eps=1e-7):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.eps = eps

#     def forward(self, input, target):
#         logit = F.sigmoid(input)
#         logit = logit.clamp(self.eps, 1. - self.eps)
#         pt = logit * target + (1- logit) * (1 - target)

#         loss = -1 * ((1-pt) ** self.gamma) * torch.log(pt) # cross entropy

#         return loss.mean()

In [ ]:
# reproduceable

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [ ]:
label_csv = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv', )
label_csv = label_csv[(label_csv.BraTS21ID!=109) & (label_csv.BraTS21ID!=123) & (label_csv.BraTS21ID!=709)]

traincsv_path = './train_label.csv'
valcsv_path = './test_label.csv'
train_portion = 0.8

maskdilate_kernelsize=5

img_dir = '../input/imagemask-voxel/train_voxel/train_voxel/'
mask_dir = '../input/imagemask-voxel/trainmask_voxel/trainmask_voxel/'

sample_indexs = np.arange(len(label_csv))
np.random.shuffle(sample_indexs)
train_indexs = sample_indexs[:int(len(label_csv) * train_portion)]
val_indexs = sample_indexs[int(len(label_csv) * train_portion):]
print(len(set(train_indexs)), len(set(val_indexs)), len(set(np.concatenate((train_indexs, val_indexs)))))

#%%

train_csv = label_csv.iloc[np.array(train_indexs)]
val_csv = label_csv.iloc[np.array(val_indexs)]

train_csv.to_csv(traincsv_path, index=False)
val_csv.to_csv(valcsv_path, index=False)

In [ ]:
# # build model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # model = DenseNet3D.densenet121(pretrained=False, progress=True, ).to(device)
# model = DenseNet3D.DenseNet(growth_rate = 32, block_config = (6, 12, 24, 16), num_init_features = 64, bn_size = 4, drop_rate = 0).to(device)
# print(torch.cuda.device_count())
# model.train()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = DenseNet3D.densenet121(pretrained=False, progress=True, ).to(device)
model = Simple_3dCNN.CNN3D().to(device)
print(torch.cuda.device_count())
model.train()

In [ ]:
# # model initialize with ImageNet training result

# model_urls = {
#     'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
#     'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
#     'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',
#     'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',
# }
# state_dict = torch.hub.load_state_dict_from_url(model_urls['densenet121'], progress=True, map_location=device)

# pattern = re.compile(
#     r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')

# for key in list(state_dict.keys()):
#     res = pattern.match(key)
#     if res:
#         new_key = res.group(1) + res.group(2)
#         if len(state_dict[key].shape) == 4:
#             state_3d = torch.unsqueeze(state_dict[key], dim=2)
#             if state_3d.shape[-1] != 1 :
#                 state_3d = torch.cat(((state_3d,)*state_3d.shape[-1]), dim=2)
#             state_dict[new_key] = state_3d.to(device)
#             print(f'key{new_key} shape changed from {state_dict[key].shape} to{state_dict[new_key].shape}')
#         else:
#             state_dict[new_key] = state_dict[key].to(device)
#         del state_dict[key]

#     elif 'conv' in key:
#         if len(state_dict[key].shape) == 4:
#             state_3d = torch.unsqueeze(state_dict[key], dim=2)
#             if state_3d.shape[-1] != 1 :
#                 state_3d = torch.cat(((state_3d,)*state_3d.shape[-1]), dim=2)
#             print(f'key{key} shape changed from {state_dict[key].shape} to{state_3d.shape}')
#             state_dict[key] = state_3d.to(device)

# featureused_RGB = 0
# conv0_weight = torch.unsqueeze(state_dict['features.conv0.weight'][:, featureused_RGB, : ,:, :], dim=1).to(device)
# print(f"key features.conv0.weight shape changed from {state_dict['features.conv0.weight'].shape}", end='')
# state_dict['features.conv0.weight'] = torch.cat((conv0_weight, ) * 4, dim=1).to(device)
# print(f"to {state_dict['features.conv0.weight'].shape} duplicating original feature map {featureused_RGB}")

# classifierused = 0
# classifier_weight = torch.unsqueeze(state_dict['classifier.weight'][classifierused, :], dim=0).to(device)
# print(f"key fclassifier.weight shape changed from {state_dict['classifier.weight'].shape}", end='')
# state_dict['classifier.weight'] = classifier_weight.to(device)
# print(f"to {state_dict['classifier.weight'].shape} taking one dimension of  original feature map {featureused_RGB}")

# classifierused = 0
# classifier_bias = torch.tensor([state_dict['classifier.bias'][classifierused]]).to(device)
# print(f"key classifier.bias shape changed from {state_dict['classifier.bias'].shape}", end='')
# state_dict['classifier.bias'] = classifier_bias.to(device)
# print(f"to {state_dict['classifier.bias'].shape} taking one dimension of original feature map {featureused_RGB}")


# model.load_state_dict(state_dict)

In [ ]:
# def plot3view_mask_4C(image_C1, image_C2, image_C3, image_C4, mask=None, title=''):

#         fig = plt.figure(figsize=(12, 25))
#         fig.suptitle(f"{title}", fontsize=24)
#         if mask is not None:
#             ax = fig.add_subplot(10 ,3, 1)
#             ax.set_title(f'mask x-y{mask.shape}')
#             ax.imshow(mask[mask.shape[0] // 2], cmap='gray')
#             ax = fig.add_subplot(10 ,3, 2)
#             ax.set_title('mask y-z')
#             ax.imshow(mask[:, mask.shape[1] // 2], cmap='gray')
#             ax = fig.add_subplot(10 ,3, 3)
#             ax.set_title('mask z-x')
#             ax.imshow(mask[:, :, mask.shape[2] // 2], cmap='gray')

#         ax = fig.add_subplot(10 ,3, 4)
#         ax.set_title(f'FLAIR x-y{image_C1.shape}')
#         ax.imshow(image_C1[image_C1.shape[0] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 5)
#         ax.set_title('FLAIR y-z')
#         ax.imshow(image_C1[:, image_C1.shape[1] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 6)
#         ax.set_title('FLAIR z-x')
#         ax.imshow(image_C1[:, :, image_C1.shape[2] // 2], cmap='gray')

#         ax = fig.add_subplot(10 ,3, 7)
#         ax.set_title(f'T1w x-y{image_C1.shape}')
#         ax.imshow(image_C2[image_C1.shape[0] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 8)
#         ax.set_title('T1w y-z')
#         ax.imshow(image_C2[:, image_C1.shape[1] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 9)
#         ax.set_title('T1w z-x')
#         ax.imshow(image_C2[:, :, image_C1.shape[2] // 2], cmap='gray')

#         ax = fig.add_subplot(10 ,3, 10)
#         ax.set_title(f'FLAIR x-y{image_C1.shape}')
#         ax.imshow(image_C3[image_C1.shape[0] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 11)
#         ax.set_title('FLAIR y-z')
#         ax.imshow(image_C3[:, image_C1.shape[1] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 12)
#         ax.set_title('FLAIR z-x')
#         ax.imshow(image_C3[:, :, image_C1.shape[2] // 2], cmap='gray')

#         ax = fig.add_subplot(10 ,3, 13)
#         ax.set_title(f'FLAIR x-y{image_C1.shape}')
#         ax.imshow(image_C4[image_C1.shape[0] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 14)
#         ax.set_title('FLAIR y-z')
#         ax.imshow(image_C4[:, image_C1.shape[1] // 2], cmap='gray')
#         ax = fig.add_subplot(10 ,3, 15)
#         ax.set_title('FLAIR z-x')
#         ax.imshow(image_C4[:, :, image_C1.shape[2] // 2], cmap='gray')

In [ ]:
# dataloader initialize

class Image4CDataSet(Dataset):
    def __init__(self, label_csv, img_dir, mask_dir=None, maskdilate_kernelsize=3, transform=None, transform_mask=None,
                 target_transform=None, Training = True):
        
        self.img_labels = pd.read_csv(label_csv, )
        self.mask_dir = mask_dir
        self.img_dir_C1 = os.path.join(img_dir, 'FLAIR')
        self.img_dir_C2 = os.path.join(img_dir, 'T1w')
        self.img_dir_C3 = os.path.join(img_dir, 'T1wCE')
        self.img_dir_C4 = os.path.join(img_dir, 'T2w')

        self.maskdilate_kernelsize = maskdilate_kernelsize

        self.transform = transform
        self.transform_mask = transform_mask
        self.target_transform = target_transform

        self.Training = Training
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
#         print(f'__getitem__ called for id: {idx}')
              
        img_path_C1 = os.path.join(self.img_dir_C1, f'{self.img_labels.iloc[idx, 0]:05}.npy')
        img_path_C2 = os.path.join(self.img_dir_C2, f'{self.img_labels.iloc[idx, 0]:05}.npy')
        img_path_C3 = os.path.join(self.img_dir_C3, f'{self.img_labels.iloc[idx, 0]:05}.npy')
        img_path_C4 = os.path.join(self.img_dir_C4, f'{self.img_labels.iloc[idx, 0]:05}.npy')

        image_C1 = torch.from_numpy(np.load(img_path_C1)).type(torch.float)
        image_C2 = torch.from_numpy(np.load(img_path_C2)).type(torch.float)
        image_C3 = torch.from_numpy(np.load(img_path_C3)).type(torch.float)
        image_C4 = torch.from_numpy(np.load(img_path_C4)).type(torch.float)
        

        image4C = torch.cat((torch.unsqueeze(image_C1, 0),
                     torch.unsqueeze(image_C2, 0),
                     torch.unsqueeze(image_C3, 0),
                     torch.unsqueeze(image_C4, 0),), dim=0)
#         plot3view_mask_4C(image4C[0, ...], image4C[1, ...], image4C[2, ...], image4C[3, ...],
#                               None, title={f'id: {idx}; operation done: maske dilate, cutoutl, rotate, Gaussian'})

        if self.Training:
            label = torch.tensor(self.img_labels.iloc[idx, 1] *1.).type(torch.float)
        else:
            label = f'{self.img_labels.iloc[idx, 0]:05}'

        if self.transform:
            mask_path = os.path.join(self.mask_dir, f'{self.img_labels.iloc[idx, 0]:05}.npy')
            mask = torch.from_numpy(np.load(mask_path)).type(torch.float)

            assert self.maskdilate_kernelsize % 2 == 1, 'maskdilate_kernelsize must be odd'
            mask = torch.nn.functional.max_pool3d(input = mask.unsqueeze(0), kernel_size=self.maskdilate_kernelsize,
                                              stride=1, padding=int((self.maskdilate_kernelsize-1)/2), dilation=1,
                                              return_indices=False, ceil_mode=False).squeeze(0)
            
#             plot3view_mask_4C(image4C[0, ...], image4C[1, ...], image4C[2, ...], image4C[3, ...],
#                               mask, title={f'id: {idx}; operation done: None'})
#             print('None:')
#             print(torch.max(image4C[0, ...]), torch.max(image4C[1, ...]), torch.max(image4C[2, ...]), torch.max(image4C[3, ...]))
#             print(torch.min(image4C[0, ...]), torch.min(image4C[1, ...]), torch.min(image4C[2, ...]), torch.min(image4C[3, ...]))
            
            image4C = self.transform_mask(image4C, mask)
#             plot3view_mask_4C(image4C[0, ...], image4C[1, ...], image4C[2, ...], image4C[3, ...],
#                               mask, title={f'id: {idx}; operation done: maske dilate, cutout'})
            
            
#             print('Cutout:')
#             print(torch.max(image4C[0, ...]), torch.max(image4C[1, ...]), torch.max(image4C[2, ...]), torch.max(image4C[3, ...]))
#             print(torch.min(image4C[0, ...]), torch.min(image4C[1, ...]), torch.min(image4C[2, ...]), torch.min(image4C[3, ...]))
            image4C = self.transform[0](image4C)
#             plot3view_mask_4C(image4C[0, ...], image4C[1, ...], image4C[2, ...], image4C[3, ...],
#                               mask, title={f'id: {idx}; operation done: maske dilate, cutoutl, rotate'})

#             print('cutoutl, rotate:')
#             print(torch.max(image4C[0, ...]), torch.max(image4C[1, ...]), torch.max(image4C[2, ...]), torch.max(image4C[3, ...]))
#             print(torch.min(image4C[0, ...]), torch.min(image4C[1, ...]), torch.min(image4C[2, ...]), torch.min(image4C[3, ...]))
            image4C = self.transform[1](image4C)
#             image4C = self.transform[2](image4C)

#             print(torch.max(image4C[0, ...]), torch.max(image4C[1, ...]), torch.max(image4C[2, ...]), torch.max(image4C[3, ...]))
#             print(torch.min(image4C[0, ...]), torch.min(image4C[1, ...]), torch.min(image4C[2, ...]), torch.min(image4C[3, ...]))
#             plot3view_mask_4C(image4C[0, ...], image4C[1, ...], image4C[2, ...], image4C[3, ...],
#                               mask, title={f'id: {idx}; operation done: maske dilate, cutoutl, rotate, Gaussian'})
        if self.target_transform:
            label = self.target_transform(label)
            

        return image4C/255, label

In [ ]:
#transform tuple

transform_mask = transforms3D.RandomCutout_withmask(num_cut=50, percent_cut=0.25,)
# transform_train = transforms3D.Compose([
#     transforms3D.RandomRotation(degrees=15),
#     transforms3D.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
# ])
transform_train = [transforms3D.RandomRotation(degrees=15), transforms3D.GaussianBlur(kernel_size=[1, 3, 3], sigma=(0.1, 3.0))]#, transforms3D.GammaAdjust(gamma_range=(0.8, 1.2), gain_range=(0.8, 1.2))]
# !pip install elasticdeform
# import elasticdeform

# transform_train = transforms3D.GammaAdjust(gamma_range=(0.8, 1.2), gain_range=(0.8, 1.2))

In [ ]:
def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [ ]:
batch_size = 15

train_dataset = Image4CDataSet(label_csv = traincsv_path,
                                 img_dir = img_dir,
                                 mask_dir = mask_dir,
                                 maskdilate_kernelsize=maskdilate_kernelsize, transform=transform_train, transform_mask=transform_mask, Training = True)

val_dataset = Image4CDataSet(label_csv = valcsv_path,
                                 img_dir = img_dir,
                                 Training = True)

# train_dataset, val_dataset = torch.utils.data.random_split(image4c_dataset,
#         [int(train_ratio * len(image4c_dataset)), len(image4c_dataset) - int(train_ratio * len(image4c_dataset))],) #generator=torch.Generator().manual_seed(0))

# train_dataset, val_dataset = torch.utils.data.random_split(image4c_dataset, [6, 4],)

print(f'training set len:{len(train_dataset)} \n validation set len:{len(val_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True, num_workers = 2, worker_init_fn=worker_init_fn)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, drop_last = True, num_workers = 1, worker_init_fn=worker_init_fn)

print(f'with batch size {batch_size} \ntraining loader len:{len(train_dataloader)} \n validation loader len:{len(val_dataloader)}')
image, label = train_dataset[0]
print(image.shape, label)
image, label = train_dataset[1]
print(image.shape, label)
image, label = val_dataset[0]
print(image.shape, label)


In [ ]:
class Trainer:
    def __init__(
            self,
            model,
            device,
            optimizer,
            loss_fn
    ):
        self.model = model.to(device)
        self.device = device
        self.optimizer = optimizer
        self.lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=LR_DECAY)
        self.loss_fn = loss_fn

        self.best_valid_score = .0
        self.n_patience = 0
        self.lastmodel = None

        self.val_losses = []
        self.train_losses = []
        self.val_auc = []
        self.train_auc = []

    def fit(self, epochs, train_dataloader, valid_dataloader, mrimodule, patience, model_savepath):
        for n_epoch in range(1, epochs + 1):
            np.random.seed(2021 + n_epoch)
            self.info_message("EPOCH: {}", n_epoch)

            train_loss, train_auc, train_time = self.train_epoch(train_dataloader)
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_dataloader)

            self.train_losses.append(train_loss)
            self.train_auc.append(train_auc)
            self.val_losses.append(valid_loss)
            self.val_auc.append(valid_auc)

            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, train_loss, train_auc, train_time
            )

            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )

            if self.best_valid_score < valid_auc:
                self.save_model(model_savepath, n_epoch, mrimodule, valid_loss, valid_auc)
                self.info_message(
                    "auc improved from {:.4f} to {:.4f}. Saved model to '{}'",
                    self.best_valid_score, valid_auc, self.lastmodel
                )
                self.best_valid_score = valid_auc
                self.n_patience = 0
            else:
                self.n_patience += 1

            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break

    def train_epoch(self, train_dataloader):
        # print('in trainer')
        self.model.train()
        t = time.time()
        sum_loss = 0

        y_all = []
        outputs_all = []

        for step, (X, Y) in enumerate(train_dataloader, 1):
            # print(step)
            X = X.to(self.device)
            Y = Y.to(self.device)
            self.optimizer.zero_grad()
            pred = self.model(X).squeeze(1)
#             print(X.is_cuda, Y.is_cuda, next(model.parameters()).is_cuda)
            loss = self.loss_fn(pred, Y)

            loss.backward()

            sum_loss += loss.detach().item()

            self.optimizer.step()

            y_all.extend(Y.tolist())
            outputs_all.extend(torch.sigmoid(pred).tolist())

            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_dataloader), sum_loss/step, end="\r")

        self.lr_scheduler.step()
        auc = roc_auc_score(y_all, outputs_all)

        return sum_loss/len(train_dataloader), auc, int(time.time() - t)

    def valid_epoch(self, valid_dataloader):
        # print('in valdier')
        self.model.eval()
        t = time.time()
        sum_loss = 0

        y_all = []
        outputs_all = []

        with torch.no_grad():
            for step, (X, Y) in enumerate(valid_dataloader, 1):
                X = X.to(self.device)
                Y = Y.to(self.device)

                pred = self.model(X).squeeze(1)
                loss = self.loss_fn(pred, Y)
                sum_loss += loss.detach().item()

                y_all.extend(Y.tolist())
                outputs_all.extend(torch.sigmoid(pred).tolist())

                message = 'Valid Step {}/{}, valid_loss: {:.4f}'
                self.info_message(message, step, len(valid_dataloader), sum_loss/step, end="\r")

        auc = roc_auc_score(y_all, outputs_all)

        return sum_loss/len(valid_dataloader), auc, int(time.time() - t)

    def save_model(self, model_savepath, n_epoch, mrimodule, loss, auc):
        self.lastmodel = f"{model_savepath}/{mrimodule}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )

    def display_plots(self, mrimodule):
        plt.figure(figsize=(10,5))
        plt.title("{}: Training and Validation Loss".format(mrimodule))
        plt.plot(self.val_losses,label="val")
        plt.plot(self.train_losses,label="train")
        plt.xlabel("iterations")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()
        plt.close()

        plt.figure(figsize=(10,5))
        plt.title("{}: Training and Validation AUC-ROC".format(mrimodule))
        plt.plot(self.val_auc,label="val")
        plt.plot(self.train_auc,label="train")
        plt.xlabel("iterations")
        plt.ylabel("AUC")
        plt.legend()
        plt.show()
        plt.close()

    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

In [ ]:
N_EPOCHS = 20
LEARNING_RATE = 0.0005
LR_DECAY = 0.9

model_savepath = './'
# train, plot and save
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.BCEWithLogitsLoss()
# loss_fn = FocalLoss(gamma=2, eps=1e-7)
# loss_fn_valid = torch.nn.BCEWithLogitsLoss(weight=torch.ones(1).to(device))


model.to(device)

trainer = Trainer(model, device, optimizer, loss_fn)

trainer.fit(N_EPOCHS, train_dataloader=train_dataloader, valid_dataloader=val_dataloader,
            mrimodule="4C", patience=10 , model_savepath=model_savepath)

trainer.display_plots("4C")

print(f'best module achieved: {trainer.lastmodel}')
